# TARS Repo Functional Test Notebook

This Colab notebook validates the repository setup, runs tests, and executes both an offline mock analysis and an optional live Gemini analysis.

## 1) Clone and install
Run this first in Colab.

In [ ]:
!git clone https://github.com/<your-org-or-user>/tars.git
%cd tars
!python -m pip install --upgrade pip
!pip install -e .

## 2) Run unit tests

In [ ]:
!python -m unittest discover -s tests

## 3) Offline functionality test (no Gemini key needed)
This monkeypatches `GeminiEvaluator` so you can verify report generation end-to-end.

In [ ]:
import json
from pathlib import Path

from tars_analyzer import analyzer
from tars_analyzer.models import GeminiEvaluation

class FakeEvaluator:
    def __init__(self, model='gemini-2.0-flash'):
        self.model = model

    def evaluate(self, conversation):
        rank = int(conversation.conversation_id.split('-')[-1])
        base = 6.0 + rank
        return GeminiEvaluation(
            helpfulness=base,
            correctness=base,
            proactivity=base,
            user_satisfaction=base,
            confidence=8.5,
            notes='mock evaluation'
        )

sample_jsonl = '\n'.join([
    json.dumps({
        'conversation_id': 'conv-0',
        'timestamp': '2026-01-01T10:00:00Z',
        'turns': [
            {'role': 'human', 'content': 'Can you plan a trip?'},
            {'role': 'agent', 'content': 'Sure, here is a rough plan.'}
        ]
    }),
    json.dumps({
        'conversation_id': 'conv-1',
        'timestamp': '2026-01-02T10:00:00Z',
        'turns': [
            {'role': 'human', 'content': 'Can you improve the budget detail?'},
            {'role': 'agent', 'content': 'Absolutely, here is a detailed budget with assumptions.'}
        ]
    })
])

Path('tmp').mkdir(exist_ok=True)
input_path = Path('tmp/mock_conversations.jsonl')
output_dir = Path('tmp/mock_output')
input_path.write_text(sample_jsonl)

original = analyzer.GeminiEvaluator
analyzer.GeminiEvaluator = FakeEvaluator
try:
    report = analyzer.analyze_conversations(input_path, output_dir)
finally:
    analyzer.GeminiEvaluator = original

print('Trend:', report['trend_label'])
print('Average score:', report['average_score'])
print('Report files:', list(output_dir.iterdir()))

In [ ]:
!cat tmp/mock_output/report.md

## 4) Optional: Live Gemini test
Run this only if you have a Gemini API key.

In [ ]:
import os
from getpass import getpass

if not os.getenv('GEMINI_API_KEY'):
    os.environ['GEMINI_API_KEY'] = getpass('Enter GEMINI_API_KEY: ')

In [ ]:
!tars-analyze examples/conversations.jsonl --out output --model gemini-2.0-flash
!cat output/report.md